In [61]:
import pandas as pd
import numpy as np
import yfinance as yf
import os
import Strategy2 as st
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [5]:
folder_path = r"C:\Users\jason\Desktop\Factor program\financial statements\data_new"
file_name = '台灣公司上下市.feather'
file_name = os.path.join(folder_path,file_name)
df_company= pd.read_feather(file_name)
df_company=df_company.set_index(['代號','名稱','年/月'])[['是否上市']].dropna()

In [7]:
folder_path = r"C:\Users\jason\Desktop\Factor program\financial statements\data_new"
file_name ='台股月報酬調整後_改2.feather'
file_name = os.path.join(folder_path,file_name)
ret= pd.read_feather(file_name)
ret.set_index(['代號', '名稱', '年/月'], inplace=True)
ret['ME'] = ret['市值(百萬元)']*1000
ret['報酬率％_月'] = ret['報酬率％_月']*0.01
ret = ret[['報酬率％_月', 'ME']]
ret = ret.sort_index(level=['代號', '年/月'])
ret = ret[ret.index.isin(df_company.index)]
ret['報酬率％_月_f1'] = ret.groupby(['代號', '名稱'])['報酬率％_月'].shift(-1)
ret.dropna(inplace=True)
ret.columns.str.strip()

ret1 = st.yearStrategy_ret(ret.reset_index(), code=ret.index.get_level_values(0).name)


Log(price)

In [10]:
folder_path = r"C:\Users\jason\Desktop\Factor program\financial statements\data_new"
file_name ='台股月報酬調整後_改2.feather'
file_name = os.path.join(folder_path,file_name)
price= pd.read_feather(file_name)
price.columns.str.strip()
price.set_index(['代號', '名稱', '年/月'], inplace=True)
price = price[['收盤價(元)_月']].sort_index(level=['代號', '年/月'])
price = price[price.index.isin(df_company.index)]

In [11]:
log_price = np.log(price['收盤價(元)_月']).to_frame('Log(Price)')
log_price = pd.concat([log_price, ret], axis=1, join='inner').reset_index()
log_price

,代號,名稱,年/月,Log(Price),報酬率％_月,ME,報酬率％_月_f1
0,1101,台泥,1980-01-31,-0.820981,0.008032,6639000.0,0.063745
1,1101,台泥,1980-02-29,-0.776529,0.063745,7062000.0,-0.021461
2,1101,台泥,1980-03-31,-0.798508,-0.021461,6480000.0,-0.008165
3,1101,台泥,1980-04-30,-0.798508,-0.008165,6427000.0,0.000000
4,1101,台泥,1980-05-31,-0.798508,0.000000,6427000.0,-0.004116
...,...,...,...,...,...,...,...
269293,L2301,光寶,2002-05-31,3.533395,0.019022,24375000.0,-0.209433
269294,L2301,光寶,2002-06-30,3.303217,-0.209433,19063000.0,0.176473
269295,L2301,光寶,2002-07-31,3.465736,0.176473,22427000.0,0.031252
269296,L2301,光寶,2002-08-31,3.496508,0.031252,23128000.0,-0.193938


In [12]:
import Strategy2 as st
anomaly153_ew = st.ew_TenStrategy(log_price, factor=log_price['Log(Price)'], ret=log_price['報酬率％_月_f1'], reverse=False, rf1=False)
# anomaly153_ew = st.ew_Strategy(log_price, factor=log_price['Log(Price)'], ret=log_price['報酬率％_月_f1'], reverse=True, rf1=False)
print(anomaly153_ew['HML'])
anomaly153_ew['Ten content']

p1 - p10


EW 夏普比率  EW 平均報酬  EW 報酬t檢定值  EW 最大回落
Factor     HML                                           
Log(Price) p1          0.273    0.029      6.338   -0.735
           p2          0.223    0.021      5.174   -0.748
           p3          0.178    0.016      4.141   -0.767
           p4          0.164    0.015      3.800   -0.766
           p5          0.153    0.015      3.542   -0.777
           p6          0.153    0.015      3.544   -0.798
           p7          0.132    0.013      3.061   -0.817
           p8          0.115    0.012      2.680   -0.802
           p9          0.100    0.010      2.313   -0.885
           p10         0.039    0.004      0.895   -0.985
           p1 - p10    0.387    0.025      8.995   -0.275

In [14]:
import Strategy2 as st
anomaly153_vw = st.vw_TenStrategy(log_price, factor=log_price['Log(Price)'], ret=log_price['報酬率％_月_f1'], size=log_price['ME'], reverse=False, rf1=False)
# anomaly153_vw = st.vw_Strategy(log_price, factor=log_price['Log(Price)'], ret=log_price['報酬率％_月_f1'], size=log_price['ME'], reverse=True, rf1=False)
print(anomaly153_vw['HML'])
anomaly153_vw['Ten content']

p1 - p10


VW 夏普比率  VW 平均報酬  VW 報酬t檢定值  VW 最大回落
Factor     HML                                           
Log(Price) p1          0.240    0.024      5.577   -0.645
           p2          0.172    0.016      3.986   -0.713
           p3          0.170    0.015      3.948   -0.743
           p4          0.139    0.013      3.220   -0.772
           p5          0.153    0.014      3.555   -0.765
           p6          0.135    0.013      3.146   -0.827
           p7          0.130    0.012      3.008   -0.803
           p8          0.130    0.013      3.017   -0.790
           p9          0.106    0.010      2.449   -0.886
           p10         0.070    0.007      1.617   -0.973
           p1 - p10    0.238    0.017      5.532   -0.492

Fama-Frencch 三因子模型

In [17]:
# 下載公司上市檔案
folder_path = r"C:\Users\jason\Desktop\Factor program\financial statements\data_new"
file_name = '台灣銀行定存利率.feather'
file_path = os.path.join(folder_path, file_name)

rf_all = pd.read_feather(file_path)
rf_tw = rf_all[rf_all['名稱'] == '臺銀'].set_index(['年/月'])[['一個月定存固定利率']]
rf_tw['Rf'] = rf_tw['一個月定存固定利率']*0.01
rf_tw = rf_tw[['Rf']]
rf_tw

,Rf
年/月,
1989-01-31,0.04000
1989-02-28,0.04000
1989-03-31,0.04000
1989-04-30,0.06500
1989-05-31,0.09250
...,...
2024-08-31,0.01225
2024-09-30,0.01225
2024-10-31,0.01225


In [18]:
folder_path = r"C:\Users\jason\Desktop\Factor program\financial statements\data_new"
file_name = '台灣公司上下市.feather'
file_name = os.path.join(folder_path,file_name)
df_company= pd.read_feather(file_name)
df_company=df_company.set_index(['代號','名稱','年/月'])[['是否上市']].dropna()
df_company = df_company.loc[pd.IndexSlice[:, :, '1980-01-31':], :]

In [19]:
# 下載公司財報資料
folder_path = r"C:\Users\jason\Desktop\Factor program\financial statements\data_new"
file_name = '台灣上市公司累計財報資料_改3.feather'
file_path = os.path.join(folder_path, file_name)
df = pd.read_feather(file_path)
df.columns = df.columns.str.strip()
df.set_index(['代號', '名稱', '年/月'], inplace=True)
df.sort_index(level=['代號', '年/月'], inplace=True)
df = df[df.index.isin(df_company.index)]

SMB

In [20]:
smb = ret.reset_index()
smb['port'] = smb.groupby('年/月')['ME'].transform(lambda x: pd.qcut(x, q=5, labels=range(1,6)))
vw_smb = smb.groupby(['port', '年/月']).apply(lambda x: (x['報酬率％_月_f1']*x['ME']).sum()/x['ME'].sum())
vw_smb_portret = pd.concat([vw_smb[1],vw_smb[2],vw_smb[3],vw_smb[4],vw_smb[5]],axis=1)
vw_smb_portret.columns = ['p1','p2','p3','p4','p5']
vw_smb_portret['SMB'] =  vw_smb_portret['p1'] - vw_smb_portret['p5']
vw_smb_portret = vw_smb_portret[['SMB']]
vw_smb_portret

C:\Users\jason\AppData\Local\Temp\ipykernel_14720\1932660542.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  vw_smb = smb.groupby(['port', '年/月']).apply(lambda x: (x['報酬率％_月_f1']*x['ME']).sum()/x['ME'].sum())
C:\Users\jason\AppData\Local\Temp\ipykernel_14720\1932660542.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  vw_smb = smb.groupby(['port', '年/月']).apply(lambda x: (x['報酬率％_月_f1']*x['ME']).sum()/x['ME'].sum())


,SMB
年/月,
1980-01-31,0.065699
1980-02-29,-0.011575
1980-03-31,-0.004379
1980-04-30,-0.004675
1980-05-31,-0.131997
...,...
2024-07-31,0.005878
2024-08-31,-0.000407
2024-09-30,-0.035449


#### HML

In [21]:
# 計算 B/M
be = df[['股東權益總額']].reset_index()

# 將財報資料調整為年策略
bm = st.yearStrategy_fin(be, be.columns[0])
bm = bm[['股東權益總額']].merge(ret1, left_index=True, right_index=True, how='inner')
bm['B/M'] = bm['股東權益總額']/bm['ME']
bm

,股東權益總額,代號,名稱,年/月,報酬率％_月,ME,報酬率％_月_f1,form_time,B/M
code&year,,,,,,,,,
1101198112,6569254,1101,台泥,1982-07-31,-0.065518,4715000.0,-0.009980,198112,1.393267
1101198112,6569254,1101,台泥,1982-08-31,-0.009980,5020000.0,0.079680,198112,1.308616
1101198112,6569254,1101,台泥,1982-09-30,0.079680,5420000.0,0.033210,198112,1.212039
1101198112,6569254,1101,台泥,1982-10-31,0.033210,5600000.0,-0.050002,198112,1.173081
1101198112,6569254,1101,台泥,1982-11-30,-0.050002,5320000.0,0.037591,198112,1.234822
...,...,...,...,...,...,...,...,...,...
L2301200012,7835586,L2301,光寶,2002-05-31,0.019022,24375000.0,-0.209433,200012,0.321460
L2301200012,7835586,L2301,光寶,2002-06-30,-0.209433,19063000.0,0.176473,200012,0.411036
L2301200112,9343796,L2301,光寶,2002-07-31,0.176473,22427000.0,0.031252,200112,0.416632


In [22]:
bm['port'] = bm.groupby('年/月')['B/M'].transform(lambda x: pd.qcut(x, q=5, labels=range(1,6)))
# 價值加權HML因子
vw_bm = bm.groupby(['port', '年/月']).apply(
    lambda x: (x['報酬率％_月_f1']*x['ME']).sum()/x['ME'].sum())

vw_bm_portret = pd.concat([vw_bm[1],vw_bm[2],vw_bm[3],vw_bm[4],vw_bm[5]],axis=1)
vw_bm_portret.columns = ['p1','p2','p3','p4','p5']
vw_bm_portret['HML'] =  vw_bm_portret['p5'] - vw_bm_portret['p1']
vw_bm_portret = vw_bm_portret[['HML']]
vw_bm_portret

C:\Users\jason\AppData\Local\Temp\ipykernel_14720\1746588298.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  vw_bm = bm.groupby(['port', '年/月']).apply(
C:\Users\jason\AppData\Local\Temp\ipykernel_14720\1746588298.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  vw_bm = bm.groupby(['port', '年/月']).apply(


,HML
年/月,
1982-07-31,0.040280
1982-08-31,-0.042728
1982-09-30,0.003393
1982-10-31,0.030491
1982-11-30,0.004752
...,...
2024-07-31,-0.026008
2024-08-31,0.011639
2024-09-30,-0.107979


MKT

In [23]:
# 下載大盤報酬資料
folder_path = r"C:\Users\jason\Desktop\Factor program\financial statements\data_new"
file_name = '大盤每月股價資料.feather'
file_path = os.path.join(folder_path, file_name)
rm = pd.read_feather(file_path)
rm['RM'] = rm['報酬率％_月']*0.01
rm['年/月'] = pd.to_datetime(rm['年/月'].astype(str)) + pd.offsets.MonthEnd()
rm = rm.set_index('年/月')[['RM']].shift(-1).dropna()
rm

,RM
年/月,
1980-02-29,0.031772
1980-03-31,-0.067682
1980-04-30,-0.019517
1980-05-31,-0.002190
1980-06-30,-0.066861
...,...
2024-08-31,0.003097
2024-09-30,-0.001954
2024-10-31,0.026812


In [24]:
factor = pd.concat([rf_tw, rm, vw_bm_portret, vw_smb_portret], axis=1, join='inner')
factor['MKT'] = (factor['RM'] - factor['Rf'])
factor=factor.reset_index()
factor['年/月']=pd.to_datetime(factor['年/月'])
factor=factor.set_index('年/月')

In [53]:
df_ew_alpha=anomaly153_ew['EW Return']['Log(Price)'][['p1 - p10']]
df_ew_alpha.rename(columns={'p1 - p10': 'Log(Price)'}, inplace=True)
df_ew_alpha = pd.concat([df_ew_alpha, factor], axis=1, join='inner').fillna(0)
df_ew_alpha

,Log(Price),Rf,RM,HML,SMB,MKT
年/月,,,,,,
1989-01-31,0.028225,0.04000,0.202772,0.014961,0.258338,0.162772
1989-02-28,-0.017931,0.04000,0.137616,-0.017563,0.095601,0.097616
1989-03-31,0.042522,0.04000,0.055067,-0.048697,0.221928,0.015067
1989-04-30,0.066258,0.06500,0.073552,0.125895,0.137458,0.008552
1989-05-31,-0.040112,0.09250,0.239317,-0.062985,-0.033263,0.146817
...,...,...,...,...,...,...
2024-07-31,-0.051845,0.01225,-0.036162,-0.026008,0.005878,-0.048412
2024-08-31,0.049409,0.01225,0.003097,0.011639,-0.000407,-0.009153
2024-09-30,-0.025668,0.01225,-0.001954,-0.107979,-0.035449,-0.014204


In [54]:
def get_alpha(y_column, x_column):
    # 放如模型
    y = y_column
    X = sm.add_constant(x_column)
    model = sm.OLS(y, X).fit()
    # 提取 alpha 值
    alpha = model.params['const']
    return alpha

In [59]:
df_ew_alpha[['MKT','HML', 'SMB']]

,MKT,HML,SMB
年/月,,,
1989-01-31,0.162772,0.014961,0.258338
1989-02-28,0.097616,-0.017563,0.095601
1989-03-31,0.015067,-0.048697,0.221928
1989-04-30,0.008552,0.125895,0.137458
1989-05-31,0.146817,-0.062985,-0.033263
...,...,...,...
2024-07-31,-0.048412,-0.026008,0.005878
2024-08-31,-0.009153,0.011639,-0.000407
2024-09-30,-0.014204,-0.107979,-0.035449


In [ ]:
y = df_ew_alpha['Log(Price)']                # 539×1
X = df_ew_alpha[['MKT', 'HML', 'SMB']]       # 539×3
alpha = get_alpha(y_column=y, x_column=X)


In [66]:
alpha

0.021388674871732426